In [1]:
import findspark
findspark.init()

In [2]:
%matplotlib inline
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [3]:
sc = SparkContext('local','music_spark')

In [4]:
sqlContext = SQLContext(sc)

In [6]:
def parse_line(l):
    try:
        return l.split(",")
    except:
        print('error in processing{0}'.format(l))


In [7]:
songs = sc.textFile('../../data/PA_chapter4_data/YearPredictionMSD.txt').map(lambda x : parse_line(x)).toDF()

In [8]:
songs

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string, _16: string, _17: string, _18: string, _19: string, _20: string, _21: string, _22: string, _23: string, _24: string, _25: string, _26: string, _27: string, _28: string, _29: string, _30: string, _31: string, _32: string, _33: string, _34: string, _35: string, _36: string, _37: string, _38: string, _39: string, _40: string, _41: string, _42: string, _43: string, _44: string, _45: string, _46: string, _47: string, _48: string, _49: string, _50: string, _51: string, _52: string, _53: string, _54: string, _55: string, _56: string, _57: string, _58: string, _59: string, _60: string, _61: string, _62: string, _63: string, _64: string, _65: string, _66: string, _67: string, _68: string, _69: string, _70: string, _71: string, _72: string, _73: string, _74: string, _75: string, _76: string, _77: string

In [12]:
from pyspark.mllib.regression import LabeledPoint
songs_labeled = songs.rdd.map( lambda x: LabeledPoint(x[0],x[1:]) )

In [13]:
songs_train = songs_labeled.zipWithIndex().filter(lambda x:x[1]<463715).map(lambda x:x[0])

In [14]:
songs_test = songs_labeled.zipWithIndex().filter(lambda x:x[1]>=463715).map(lambda x:x[0])

In [15]:
from pyspark.mllib.tree import RandomForest
rf = RandomForest.trainRegressor(songs_train,{},50,"auto","variance",10,32)
prediction = rf.predict(songs_test.map(lambda x:x.features))
predictionObserved = songs_test.map(lambda lp: lp.label).zip(prediction)

In [16]:
from pyspark.mllib.evaluation import RegressionMetrics
RegressionMetrics(predictionObserved).r2

-3.736708080530903